# Data Loader - H8 and MODIS

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn import preprocessing as pre
from sklearn.metrics import mean_squared_error
import os
import xarray as xr
import datetime
import netCDF4 as nc
from collections import Counter
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

In [2]:
# TODO: consider splitting script into a module

import os
import pandas as pd
import numpy as np
import xarray as xr
import datetime

def read_H8_MODIS_byDate(date, modis_res, modis_sat, data_path):
    '''
    TODO: comments
    '''
    
    H8_PARENT_DIR = 'H8_MODIS_2019-2020'
    H8_MODIS_DIR = f'H8_{modis_sat}_{modis_res}'
    H8_MODIS_FILENAME = f'H8_{modis_sat}_{modis_res}_Aus_0.05_{date}.nc'
    H8_MODIS_PATH = os.path.join(data_path, H8_PARENT_DIR, H8_MODIS_DIR, H8_MODIS_FILENAME)
    
    df = xr.open_dataset(H8_MODIS_PATH).to_dataframe()
    df = df.drop(columns=['solar_azimuth_angle','solar_zenith_angle'])
    
    return df

def read_MODIS_byDate(date, modis_res, modis_sat, data_path):
    '''
    TODO: comments
    '''
    
    MODIS_PARENT_DIR = 'MODIS_L2_Aus_2019-2020'
    MODIS_FILENAME = f'{modis_sat}_{modis_res}_Aus_0.05_{date}.nc'
    MODIS_PATH = os.path.join(data_path, MODIS_PARENT_DIR, MODIS_FILENAME)
    
    df = xr.open_dataset(MODIS_PATH).drop_dims(['corner','file']).to_dataframe()
    
    return df

def concat_rows(df_list):
    '''
    TODO: comments
    '''
    
    for i in range(1,len(df_list)):
        if list(df_list[i].columns) != list(df_list[0].columns) and (not df_list[0].empty and not df_list[i].empty):
            raise Exception('ERROR: Columns do not line up in the dataframes that are to be concatenated. Data may need to be fixed before proceeding.')
            
    return pd.concat(df_list, axis=0)

def check_lat_lon(df_H8, df_MODIS):
    '''
    TODO: comments
    '''
    
    if list(df_H8.lat) != list(df_MODIS.Latitude) or list(df_H8.lon) != list(df_MODIS.Longitude):
        raise Exception('ERROR: Lat/Lon rows do not line up between the two dataframes.')

def load_data_byDate(date_str, modis_res, data_dir):
    '''
    TODO: comments
    '''
    
    df_H8_MOD04 = read_H8_MODIS_byDate(date_str, modis_res, 'MOD04', data_dir)
    df_H8_MYD04 = read_H8_MODIS_byDate(date_str, modis_res, 'MYD04', data_dir)
    df_H8 = concat_rows([df_H8_MOD04, df_H8_MYD04])
    
    df_MOD04 = read_MODIS_byDate(date_str, modis_res, 'MOD04', data_dir)
    df_MYD04 = read_MODIS_byDate(date_str, modis_res, 'MYD04', data_dir)
    df_MODIS = concat_rows([df_MOD04, df_MYD04])
    
    check_lat_lon(df_H8, df_MODIS)
    
    return df_H8, df_MODIS

def generate_dates(years, days):
    '''
    TODO: comments
    '''
    
    for year in years:
        for month in range(1, 13):
            for day in days:
                yield datetime.date(year, month, day).strftime("%Y-%m-%d")

def load_data(dates_str, modis_res, data_dir, sample_frac=1):
    '''
    TODO: comments
    '''
    
    df_H8 = pd.DataFrame()
    df_MODIS = pd.DataFrame()
    
    for date_str in dates_str:
                
        df_H8_daily, df_MODIS_daily = load_data_byDate(date_str, modis_res, data_dir)
        
        if sample_frac != 1:
            n = len(df_H8_daily)
            sample_size = int(n * sample_frac)
            sample_idx = np.random.choice(n, size=sample_size, replace=False)
            
            df_H8_daily = df_H8_daily.iloc[sample_idx]
            df_MODIS_daily = df_MODIS_daily.iloc[sample_idx]

        df_H8 = concat_rows([df_H8, df_H8_daily])
        df_MODIS = concat_rows([df_MODIS, df_MODIS_daily])
                
    return df_H8, df_MODIS

In [3]:
PARAM_DATA_YEARS = [2019]
PARAM_TRAIN_DAYSOFMONTH = [1,10,20]
PARAM_VAL_DAYSOFMONTH = [8,16]
PARAM_HOLDOUT_DAYSOFMONTH = [28]
PARAM_MODIS_RES = 'L2'
PARAM_DATA_DIR = '/Users/dongzhanchi/Study/Unimelb/2022 S2/Pt2/data'  # change this to the directory path where your H8 and MODIS files are located

PARAM_SAMPLE_FRAC = 0.03  # fraction of data sampled into our sets, change to 1 for no further sampling
# PARAM_SAMPLE_FRAC = 1

np.random.seed(111)

df_H8_train, df_MODIS_train = load_data(generate_dates(PARAM_DATA_YEARS, PARAM_TRAIN_DAYSOFMONTH), 
                                        PARAM_MODIS_RES, 
                                        PARAM_DATA_DIR,
                                        PARAM_SAMPLE_FRAC)

df_H8_val, df_MODIS_val = load_data(generate_dates(PARAM_DATA_YEARS, PARAM_VAL_DAYSOFMONTH), 
                                    PARAM_MODIS_RES, 
                                    PARAM_DATA_DIR,
                                    PARAM_SAMPLE_FRAC)

df_H8_holdout, df_MODIS_holdout = load_data(generate_dates(PARAM_DATA_YEARS, PARAM_HOLDOUT_DAYSOFMONTH), 
                                            PARAM_MODIS_RES, 
                                            PARAM_DATA_DIR,
                                            PARAM_SAMPLE_FRAC)

In [4]:
print('train size:', len(df_H8_train))
print('val size:', len(df_H8_val))
print('hold size',len(df_H8_holdout))

# df_H8_train = df_H8_train.dropna()
# df_H8_val = df_H8_val.dropna()

# print('train size:', len(df_H8_train))
# print('val size:', len(df_H8_val))

train size: 49193
val size: 32656
hold size 14900


# Sample usage - e.g. for modelling

In [5]:
feature_cols = [
    'lat',
    'lon',
    # 'channel_0001_brf',
    'channel_0001_scaled_radiance',
    # 'channel_0002_brf',
    'channel_0002_scaled_radiance',
    # 'channel_0003_brf',
    'channel_0003_scaled_radiance',
    # 'channel_0004_brf',
    'channel_0004_scaled_radiance',
    # 'channel_0005_brf',
    'channel_0005_scaled_radiance',
    # 'channel_0006_brf',
    'channel_0006_scaled_radiance',
    'channel_0007_brightness_temperature',
    'channel_0008_brightness_temperature',
    'channel_0009_brightness_temperature',
    'channel_0010_brightness_temperature',
    'channel_0011_brightness_temperature',
    'channel_0012_brightness_temperature',
    'channel_0013_brightness_temperature',
    'channel_0014_brightness_temperature',
    'channel_0015_brightness_temperature',
    'channel_0016_brightness_temperature',
]

response_col = 'AOD_550_Dark_Target_Deep_Blue_Combined'

X_train = df_H8_train[feature_cols]
y_train = df_MODIS_train[response_col]
X_val = df_H8_val[feature_cols]
y_val = df_MODIS_val[response_col]

train = pd.concat([X_train, y_train],axis=1)
test =  pd.concat([X_val, y_val],axis=1)



X_holdout = df_H8_holdout[feature_cols]
y_holdout = df_MODIS_holdout[response_col]


holdout = pd.concat([X_holdout, y_holdout],axis=1)

In [6]:
print(train.shape)
train.head()

(49193, 19)


,lat,lon,channel_0001_scaled_radiance,channel_0002_scaled_radiance,channel_0003_scaled_radiance,channel_0004_scaled_radiance,channel_0005_scaled_radiance,channel_0006_scaled_radiance,channel_0007_brightness_temperature,channel_0008_brightness_temperature,channel_0009_brightness_temperature,channel_0010_brightness_temperature,channel_0011_brightness_temperature,channel_0012_brightness_temperature,channel_0013_brightness_temperature,channel_0014_brightness_temperature,channel_0015_brightness_temperature,channel_0016_brightness_temperature,AOD_550_Dark_Target_Deep_Blue_Combined
sounding,,,,,,,,,,,,,,,,,,,
5186,-27.094444,139.511383,0.159180,0.159180,0.274414,0.383789,0.539062,0.382812,336.1875,244.9375,253.0625,260.5625,315.1250,288.8125,325.5625,323.9375,314.1875,286.5625,0.099
9478,-19.180023,139.592560,0.129883,0.119141,0.161133,0.265625,0.320312,0.179688,328.9375,253.5625,266.6250,273.8125,315.1875,289.8750,319.6250,318.2500,310.7500,287.5000,0.076
16433,-22.822483,122.261169,0.096680,0.086914,0.139648,0.208984,0.320312,0.242188,328.4375,241.8750,250.1875,258.0000,308.8125,284.9375,316.6875,313.8125,304.6250,281.7500,0.018
1481,-33.870541,150.778870,0.125000,0.111328,0.100586,0.249023,0.203125,0.093750,313.5625,241.5000,252.9375,261.0625,301.8125,273.4375,305.5000,303.8750,297.5000,277.5625,0.077
7793,-25.254416,139.820129,0.156250,0.148438,0.247070,0.326172,0.500000,0.312500,330.8125,251.6250,260.1875,265.7500,313.1250,286.8750,319.8125,318.0625,309.7500,284.7500,0.137


In [7]:
holdout.shape

(14900, 19)

# 上面的都是vito的

In [8]:
import numpy as np
import netCDF4 as nc
import pandas as pd
import xarray as xr
from collections import Counter



def load_landcover(path):
    landcover = xr.open_dataset(path)
    
    # drop all the dims except lat and lon
    landcover_df = landcover.drop_dims(['Num_IGBP_Classes_MOD12C1','Num_UMD_Classes_MOD12C1','Num_LAI_FPAR_Classes_MOD12C1','latbnd','lonbnd'])
    landcover_df = landcover_df.to_dataframe()

    #landcover_df = landcover_df.dropna()
    #landcover_df = landcover_df.drop_duplicates()
    landcover_df = landcover_df.reset_index()


    useless_cols = ['Majority_Land_Cover_Type_2','Majority_Land_Cover_Type_2_Assessment','Majority_Land_Cover_Type_3',
                    'Majority_Land_Cover_Type_3_Assessment','Majority_Land_Cover_Type_1_Assessment']
    
    landcover_df = landcover_df.drop(columns = useless_cols)
    
    
    return landcover_df    



def load_train(H8_MOD04_path,MOD04_L2_path):
    # load H8_MOD04
    H8_MOD04 = xr.open_dataset(H8_MOD04_path)
    H8_MOD04_df = H8_MOD04.to_dataframe()
    
    
    # load MOD04_L2
    MOD04_L2 = nc.Dataset(MOD04_L2_path)

    response = MOD04_L2['AOD_550_Dark_Target_Deep_Blue_Combined'][:]
    AOD_ds = pd.Series(response,name = 'AOD_550_Dark_Target_Deep_Blue_Combined')
    AOD_df = AOD_ds.to_frame()
    
    # combine the training set 
    train_df = pd.concat([H8_MOD04_df,AOD_df],axis=1)
    
    return train_df


def build_class_file(file, landcover_path):
    '''
    TODO: comments
    '''
    # data in training set
    landcover = nc.Dataset(landcover_path)
    lat = np.array(file['lat'])
    lon = np.array(file['lon'])

    # data in landcover data set
    latbnd = landcover.variables['latbnd']
    lonbnd = landcover.variables['lonbnd']
    latbnd_arr = sorted(latbnd[:]) # grid-bnd of lat, decreasing
    lonbnd_arr = lonbnd[:]         # grid-bnd of lon, increasing
    
    
    lat_class = np.searchsorted(latbnd_arr, lat,side='left')
    ny = len(latbnd_arr)
    lat_class  = ny - np.searchsorted(latbnd_arr, lat,side='left')
    
    lon_class = np.searchsorted(lonbnd_arr,lon,side='left')
    file['lat_class'] = lat_class
    file['lon_class'] = lon_class
    
    return file


def combine_tables_by_class(train,landcover):
    '''
    TODO: comments
    '''
    result = pd.merge(train, landcover, on=['lat_class', 'lon_class'],how = 'left')
    result = result.dropna()
    
    return result

In [9]:
landcover2019_path = '/Users/dongzhanchi/Study/Unimelb/2022 S2/Pt2/input/MAST90106-7_2022_Group3/MCD12C1.A2019001.006.2020220162300.nc'
landcover2019 = load_landcover(landcover2019_path)
landcover_df = build_class_file(landcover2019, landcover2019_path)

train_df = build_class_file(train, landcover2019_path)
train = combine_tables_by_class(train_df,landcover_df)
useless_cols = ['lat_x','lat_y','lon_x','lon_y']
train = train.drop(columns = useless_cols)

y_train = train['AOD_550_Dark_Target_Deep_Blue_Combined']
X_train = train.drop(columns = ['AOD_550_Dark_Target_Deep_Blue_Combined'])
y_train = train[['AOD_550_Dark_Target_Deep_Blue_Combined']]

In [10]:
y_train.shape

(47914, 1)

In [11]:
test_df = build_class_file(test, landcover2019_path)
test = combine_tables_by_class(test_df,landcover_df)
useless_cols = ['lat_x','lat_y','lon_x','lon_y']
test = test.drop(columns = useless_cols)


X_val = test.drop(columns = ['AOD_550_Dark_Target_Deep_Blue_Combined'])
X_val.head()

,channel_0001_scaled_radiance,channel_0002_scaled_radiance,channel_0003_scaled_radiance,channel_0004_scaled_radiance,channel_0005_scaled_radiance,channel_0006_scaled_radiance,channel_0007_brightness_temperature,channel_0008_brightness_temperature,channel_0009_brightness_temperature,channel_0010_brightness_temperature,channel_0011_brightness_temperature,channel_0012_brightness_temperature,channel_0013_brightness_temperature,channel_0014_brightness_temperature,channel_0015_brightness_temperature,channel_0016_brightness_temperature,lat_class,lon_class,Majority_Land_Cover_Type_1
0,0.114258,0.099609,0.153320,0.195312,0.289062,0.218750,326.2500,249.7500,260.1875,267.3125,308.9375,283.8750,315.3125,314.5000,307.9375,285.4375,215,380,7.0
1,0.109375,0.099609,0.144531,0.225586,0.304688,0.203125,321.1875,237.6250,249.0625,259.5000,307.6250,279.1250,311.9375,309.9375,302.0000,279.2500,435,341,7.0
2,0.123047,0.105469,0.145508,0.240234,0.312500,0.218750,321.4375,241.8750,255.1250,265.2500,307.8750,281.1250,314.5000,314.0625,308.1250,284.1250,379,308,7.0
3,0.117188,0.099609,0.132812,0.217773,0.250000,0.164062,315.8750,238.6875,252.4375,262.8125,302.1875,276.3125,308.5000,307.6875,301.9375,279.7500,411,275,7.0
4,0.136719,0.126953,0.192383,0.280273,0.406250,0.273438,321.1250,246.8125,258.2500,267.2500,309.3125,280.4375,313.4375,312.7500,306.9375,283.0000,410,402,7.0


In [12]:
y_val = test[['AOD_550_Dark_Target_Deep_Blue_Combined']]

In [13]:
y_val.shape

(32636, 1)

In [14]:
holdout_df = build_class_file(holdout, landcover2019_path)
holdout = combine_tables_by_class(holdout_df,landcover_df)
useless_cols = ['lat_x','lat_y','lon_x','lon_y']
holdout = holdout.drop(columns = useless_cols)




X_holdout = holdout.drop(columns = ['AOD_550_Dark_Target_Deep_Blue_Combined'])
y_holdout = holdout[['AOD_550_Dark_Target_Deep_Blue_Combined']]



In [15]:
X_holdout.head()

,channel_0001_scaled_radiance,channel_0002_scaled_radiance,channel_0003_scaled_radiance,channel_0004_scaled_radiance,channel_0005_scaled_radiance,channel_0006_scaled_radiance,channel_0007_brightness_temperature,channel_0008_brightness_temperature,channel_0009_brightness_temperature,channel_0010_brightness_temperature,channel_0011_brightness_temperature,channel_0012_brightness_temperature,channel_0013_brightness_temperature,channel_0014_brightness_temperature,channel_0015_brightness_temperature,channel_0016_brightness_temperature,lat_class,lon_class,Majority_Land_Cover_Type_1
0,0.138672,0.138672,0.208984,0.330078,0.398438,0.218750,332.7500,250.1875,261.4375,268.250,320.7500,290.0625,326.1250,325.6875,317.9375,288.9375,389,115,12.0
1,0.162109,0.134766,-0.000977,0.105469,0.224609,0.154297,306.1250,240.0000,251.4375,259.750,292.1875,269.5000,295.6250,294.0625,289.2500,273.1250,359,473,7.0
2,0.196289,0.161133,0.000977,0.089844,0.277344,0.150391,321.3750,249.2500,261.3750,270.125,312.2500,281.9375,316.8125,317.0000,311.9375,285.7500,460,173,12.0
3,0.095703,0.083008,0.132812,0.199219,0.335938,0.234375,329.9375,246.2500,258.7500,268.625,316.8125,286.4375,324.0000,323.7500,317.6250,290.0625,347,223,7.0
4,0.213867,0.212891,-0.000977,0.265625,0.324219,0.210938,316.3125,244.8125,252.5000,259.750,297.6250,273.5000,299.8750,299.6250,294.6250,277.4375,276,507,7.0


In [16]:
y_holdout

,AOD_550_Dark_Target_Deep_Blue_Combined
0,0.017
1,0.018
2,0.063
3,0.018
4,0.054
...,...
14895,0.119
14896,0.018
14897,0.016
14898,0.103


# Use randomforest

In [17]:
from sklearn.ensemble import RandomForestRegressor as rfr
from sklearn.model_selection import cross_val_score

In [18]:
from sklearn.preprocessing import StandardScaler
ss_x = StandardScaler()
X_train = ss_x.fit_transform(X_train)
X_holdout = ss_x.transform(X_holdout)
X_val = ss_x.transform(X_val)

# ss_y = StandardScaler()
# y_train = ss_y.fit_transform(pd.DataFrame(y_train))
# y_holdout = ss_y.fit_transform(pd.DataFrame(y_holdout))
# y_val = ss_y.fit_transform(pd.DataFrame(y_val))

In [20]:
rf = rfr(n_estimators = 100, max_features = 'sqrt', max_depth = 50, random_state = 18).fit(X_train, y_train)

/var/folders/rn/r6mn6s4n297cf0gwwrlbx0ch0000gn/T/ipykernel_61313/4281158340.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf = rfr(n_estimators = 100, max_features = 'sqrt', max_depth = 50, random_state = 18).fit(X_train, y_train)


In [21]:
rf_prediction = rf.predict(X_val)

In [22]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_val, rf_prediction)
rmse = mse**.5
print(mse)
print(rmse)

0.0032046041868485867
0.056609223513916763


In [23]:
from sklearn.metrics import r2_score
print("R_squared is：", r2_score(y_val, rf_prediction))

R_squared is： 0.3156794485722981


## for holdout 

In [25]:
rf_holdout = rf.predict(X_holdout)
mse = mean_squared_error(y_holdout, rf_holdout)
rmse = mse**.5

In [26]:
print(mse)
print(rmse)

0.0024828923133365243
0.04982862945472737


In [27]:
from sklearn.metrics import r2_score
print('for randomforest in holdoutset')
print("R_squared is：", r2_score(y_holdout, rf_holdout))
print('MSE is: ', mse)
print('RMSE is :',rmse)

for randomforest in holdoutset
R_squared is： 0.292059516811746
MSE is:  0.0024828923133365243
RMSE is : 0.04982862945472737
